In [3]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census

# Census API Key
from config import api_key
c = Census(api_key, year=2016)

In [6]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E","B25058_001E","B25077_001E"), {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B25058_001E": "average rent",
                                      "B25077_001E": "Median Home Value"
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["Zipcode", "Name","Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate","average rent",]]

# Visualize
print(len(census_pd))
census_pd.head()

33120


,Zipcode,Name,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,average rent
0,01001,ZCTA5 01001,17423.0,45.0,56714.0,30430.0,1462.0,8.391207,835.0
1,01002,ZCTA5 01002,29970.0,23.2,48923.0,26072.0,8351.0,27.864531,1108.0
2,01003,ZCTA5 01003,11296.0,19.9,2499.0,3829.0,54.0,0.478045,1150.0
3,01005,ZCTA5 01005,5228.0,44.1,70568.0,32169.0,230.0,4.399388,761.0
4,01007,ZCTA5 01007,14888.0,42.5,80502.0,36359.0,1410.0,9.470715,787.0


In [3]:
# Save as a csv
# Note to avoid any issues later, use encoding="utf-8"
census_pd.to_csv("census_data.csv", encoding="utf-8", index=False)